# Chapter 7: 채팅 애플리케이션 구축
## Azure OpenAI API 빠른 시작


## 개요
이 노트북은 [OpenAI](notebook-openai.ipynb) 서비스에 액세스하는 노트북도 포함하는 [Azure OpenAI 샘플 저장소](https://github.com/Azure/azure-openai-samples?WT.mc_id=academic-105485-koreyst)에서 가져왔습니다.

Python OpenAI API는 약간의 수정으로 Azure OpenAI와도 작동합니다. 차이점에 대해 자세히 알아보려면 여기를 참조하세요: [Python으로 OpenAI와 Azure OpenAI 엔드포인트 간 전환 방법](https://learn.microsoft.com/azure/ai-services/openai/how-to/switching-endpoints?WT.mc_id=academic-109527-jasmineg)

더 많은 빠른 시작 예제는 공식 [Azure OpenAI 빠른 시작 문서](https://learn.microsoft.com/azure/cognitive-services/openai/quickstart?pivots=programming-language-studio&WT.mc_id=academic-105485-koreyst)를 참조하세요.


## 목차  

[개요](../../../../07-building-chat-applications/python)  
[Azure OpenAI 서비스 시작하기](../../../../07-building-chat-applications/python)  
[첫 번째 프롬프트 만들기](../../../../07-building-chat-applications/python)  

[사용 사례](../../../../07-building-chat-applications/python)    
[1. 텍스트 요약](../../../../07-building-chat-applications/python)  
[2. 텍스트 분류](../../../../07-building-chat-applications/python)  
[3. 새로운 제품 이름 생성](../../../../07-building-chat-applications/python)  
[4. 분류기 미세 조정](../../../../07-building-chat-applications/python)  
[5. 임베딩](../../../../07-building-chat-applications/python)

[참고 문헌](../../../../07-building-chat-applications/python)


### Azure OpenAI 서비스 시작하기

새 고객은 Azure OpenAI 서비스에 대한 [액세스 신청](https://aka.ms/oai/access?WT.mc_id=academic-105485-koreyst)을 해야 합니다.  
승인이 완료되면 고객은 Azure 포털에 로그인하여 Azure OpenAI 서비스 리소스를 생성하고 스튜디오를 통해 모델을 실험할 수 있습니다.  

[빠르게 시작하는 데 유용한 자료](https://techcommunity.microsoft.com/blog/educatordeveloperblog/azure-openai-service-is-now-generally-available/3719177?WT.mc_id=academic-105485-koreyst)


### 첫 번째 프롬프트 작성하기  
이 짧은 연습은 간단한 작업인 "요약"을 위해 OpenAI 모델에 프롬프트를 제출하는 기본적인 소개를 제공합니다.


**단계**:  
1. Python 환경에 OpenAI 라이브러리를 설치하세요  
2. 표준 도우미 라이브러리를 불러오고 생성한 OpenAI 서비스에 대한 일반적인 OpenAI 보안 자격 증명을 설정하세요  
3. 작업에 적합한 모델을 선택하세요  
4. 모델을 위한 간단한 프롬프트를 만드세요  
5. 모델 API에 요청을 제출하세요!


### 1. OpenAI 설치하기


  > [!참고] 이 단계는 Codespaces에서 실행하거나 Devcontainer 내에서 실행하는 경우 필요하지 않습니다


In [ ]:
%pip install openai python-dotenv

### 2. 도우미 라이브러리 가져오기 및 자격 증명 인스턴스화하기


In [ ]:
import os
from openai import AzureOpenAI
import numpy as np
from dotenv import load_dotenv
load_dotenv()

#validate data inside .env file

client = AzureOpenAI(
  api_key=os.environ['AZURE_OPENAI_API_KEY'],  # this is also the default, it can be omitted
  api_version = "2023-05-15"
  )

### 3. 적합한 모델 찾기  
GPT-3.5-turbo 또는 GPT-4 모델은 자연어를 이해하고 생성할 수 있습니다. 이 서비스는 각기 다른 작업에 적합한 다양한 수준의 성능과 속도를 가진 네 가지 모델 기능을 제공합니다. 

[Azure OpenAI models](https://learn.microsoft.com/azure/cognitive-services/openai/concepts/models?WT.mc_id=academic-105485-koreyst)


In [ ]:
# Select the General Purpose curie model for text
model = os.environ['AZURE_OPENAI_DEPLOYMENT']

## 4. 프롬프트 설계  

"대규모 언어 모델의 마법은 방대한 양의 텍스트에 대해 이 예측 오류를 최소화하도록 훈련됨으로써, 모델이 이러한 예측에 유용한 개념들을 학습하게 된다는 점입니다. 예를 들어, 모델은 다음과 같은 개념들을 학습합니다"(1):

* 철자법
* 문법 작동 방식
* 바꾸어 말하는 방법
* 질문에 답하는 방법
* 대화를 이어가는 방법
* 여러 언어로 글을 쓰는 방법
* 코딩하는 방법
* 기타 등등

#### 대규모 언어 모델을 제어하는 방법  
"대규모 언어 모델에 입력되는 모든 것 중에서, 가장 영향력이 큰 것은 단연 텍스트 프롬프트입니다(1).

대규모 언어 모델은 몇 가지 방법으로 출력 생성을 유도할 수 있습니다:

- 지시: 모델에게 원하는 바를 말하기
- 완성: 원하는 시작 부분을 모델이 완성하도록 유도하기
- 시범: 모델에게 원하는 바를 보여주기, 다음 중 하나로:
- 프롬프트 내 몇 가지 예시
- 미세 조정 훈련 데이터셋 내 수백 또는 수천 개의 예시



#### 프롬프트를 만드는 세 가지 기본 지침:

**보여주고 말하기**. 지시, 예시 또는 두 가지의 조합을 통해 원하는 바를 명확히 하세요. 모델에게 항목 목록을 알파벳 순으로 정렬하거나 문단을 감성별로 분류하도록 하려면, 그것이 원하는 바임을 보여주세요.

**양질의 데이터를 제공하기**. 분류기를 만들거나 모델이 패턴을 따르도록 하려면 충분한 예시가 있어야 합니다. 예시를 반드시 교정하세요 — 모델은 보통 기본적인 철자 오류를 간파하고 응답을 주지만, 이것이 의도된 것이라고 가정할 수도 있으며 응답에 영향을 줄 수 있습니다.

**설정을 확인하기**. temperature와 top_p 설정은 모델이 응답을 생성할 때 얼마나 결정론적인지를 제어합니다. 정답이 하나뿐인 응답을 요청할 때는 이 값을 낮게 설정하는 것이 좋습니다. 더 다양한 응답을 원할 때는 이 값을 높게 설정할 수 있습니다. 사람들이 이 설정에서 가장 많이 하는 실수는 이것이 "영리함"이나 "창의성"을 조절한다고 생각하는 것입니다.


출처: https://learn.microsoft.com/azure/ai-services/openai/overview


이미지는 당신의 첫 번째 텍스트 프롬프트를 생성하고 있습니다!


### 5. 제출하세요!


In [ ]:
# Create your first prompt
text_prompt = "Should oxford commas always be used?"

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":text_prompt},])

response.choices[0].message.content

### 같은 호출을 반복하면 결과는 어떻게 비교되나요?


In [ ]:

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":text_prompt},])

response.choices[0].message.content

## Summarize Text  
#### Challenge  
텍스트 구절 끝에 'tl;dr:'을 추가하여 텍스트를 요약합니다. 모델이 추가 지시 없이 여러 작업을 수행하는 방법을 이해하는 방식을 주목하세요. tl;dr보다 더 설명적인 프롬프트로 실험하여 모델의 동작을 수정하고 받는 요약을 맞춤화할 수 있습니다(3).  

최근 연구에서는 대규모 텍스트 코퍼스에 대한 사전 학습과 특정 작업에 대한 미세 조정을 통해 많은 NLP 작업과 벤치마크에서 상당한 성과를 보여주었습니다. 일반적으로 아키텍처는 작업에 구애받지 않지만, 이 방법은 여전히 수천 또는 수만 개의 작업별 미세 조정 데이터셋을 필요로 합니다. 반면, 인간은 일반적으로 몇 가지 예시나 간단한 지시만으로 새로운 언어 작업을 수행할 수 있는데, 이는 현재 NLP 시스템이 여전히 크게 어려워하는 부분입니다. 여기서는 언어 모델의 규모를 확장하는 것이 작업에 구애받지 않는 소수 샷 성능을 크게 향상시키며, 때로는 이전 최첨단 미세 조정 접근법과 경쟁할 수 있음을 보여줍니다.  

Tl;dr


# 여러 사용 사례를 위한 연습  
1. 텍스트 요약  
2. 텍스트 분류  
3. 새로운 제품 이름 생성  
4. 임베딩  
5. 분류기 미세 조정


In [ ]:
prompt = "Recent work has demonstrated substantial gains on many NLP tasks and benchmarks by pre-training on a large corpus of text followed by fine-tuning on a specific task. While typically task-agnostic in architecture, this method still requires task-specific fine-tuning datasets of thousands or tens of thousands of examples. By contrast, humans can generally perform a new language task from only a few examples or from simple instructions - something that current NLP systems still largely struggle to do. Here we show that scaling up language models greatly improves task-agnostic, few-shot performance, sometimes even reaching competitiveness with prior state-of-the-art fine-tuning approaches.\n\ntl;dr"


In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

response.choices[0].message.content

## 텍스트 분류  
#### 도전 과제  
추론 시 제공된 카테고리로 항목을 분류합니다. 다음 예제에서는 프롬프트(*playground_reference)에 분류할 텍스트와 카테고리를 모두 제공합니다. 

고객 문의: 안녕하세요, 제 노트북 키보드의 키 중 하나가 최근에 고장 나서 교체가 필요합니다:

분류된 카테고리:


In [ ]:
prompt = "Classify the following inquiry into one of the following: categories: [Pricing, Hardware Support, Software Support]\n\ninquiry: Hello, one of the keys on my laptop keyboard broke recently and I'll need a replacement:\n\nClassified category:"
print(prompt)

In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

response.choices[0].message.content

## 새로운 제품 이름 생성
#### 도전 과제
예시 단어에서 제품 이름을 만드세요. 여기에는 우리가 이름을 생성하려는 제품에 대한 정보가 프롬프트에 포함되어 있습니다. 또한 우리가 받고자 하는 패턴을 보여주기 위해 유사한 예시를 제공합니다. 무작위성과 더 혁신적인 응답을 높이기 위해 온도 값을 높게 설정했습니다.

제품 설명: 가정용 밀크셰이크 제조기  
시드 단어: 빠른, 건강한, 컴팩트  
제품 이름: HomeShaker, Fit Shaker, QuickShake, Shake Maker

제품 설명: 모든 발 크기에 맞는 신발 한 켤레  
시드 단어: 적응형, 맞춤형, 옴니핏


In [ ]:
prompt = "Product description: A home milkshake maker\nSeed words: fast, healthy, compact.\nProduct names: HomeShaker, Fit Shaker, QuickShake, Shake Maker\n\nProduct description: A pair of shoes that can fit any foot size.\nSeed words: adaptable, fit, omni-fit."

print(prompt)

In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt}])

response.choices[0].message.content

## 임베딩
이 섹션에서는 임베딩을 검색하고 단어, 문장 및 문서 간의 유사성을 찾는 방법을 보여줍니다. 다음 노트북을 실행하려면 `text-embedding-ada-002`를 기본 모델로 사용하는 모델을 배포하고 .env 파일 내에 `AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT` 변수를 사용하여 해당 배포 이름을 설정해야 합니다.


### 모델 분류 - 임베딩 모델 선택하기

**모델 분류**: {family} - {capability} - {input-type} - {identifier}  

{family}     --> text-embedding  (임베딩 계열)  
{capability} --> ada             (다른 모든 임베딩 모델은 2024년에 폐기될 예정)  
{input-type} --> n/a             (검색 모델에만 지정됨)  
{identifier} --> 002             (버전 002)  

model = 'text-embedding-ada-002'


  > [!참고] 이 단계를 수행하는 것은 Codespaces에서 또는 Devcontainer 내에서 이 노트북을 실행하는 경우 필요하지 않습니다


In [ ]:
# Dependencies for embeddings_utils
%pip install matplotlib plotly scikit-learn pandas

In [ ]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

In [ ]:
text = 'the quick brown fox jumped over the lazy dog'
model= os.environ['AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT']
client.embeddings.create(input='[text]', model=model).data[0].embedding

In [ ]:

# compare several words
automobile_embedding  = client.embeddings.create(input='automobile', model=model).data[0].embedding
vehicle_embedding     = client.embeddings.create(input='vehicle', model=model).data[0].embedding
dinosaur_embedding    = client.embeddings.create(input='dinosaur', model=model).data[0].embedding
stick_embedding       = client.embeddings.create(input='stick', model=model).data[0].embedding

print(cosine_similarity(automobile_embedding, vehicle_embedding))
print(cosine_similarity(automobile_embedding, dinosaur_embedding))
print(cosine_similarity(automobile_embedding, stick_embedding))

## cnn daily news 데이터셋의 기사 비교
출처: https://huggingface.co/datasets/cnn_dailymail


In [ ]:
import pandas as pd
cnn_daily_articles = ['BREMEN, Germany -- Carlos Alberto, who scored in FC Porto\'s Champions League final victory against Monaco in 2004, has joined Bundesliga club Werder Bremen for a club record fee of 7.8 million euros ($10.7 million). Carlos Alberto enjoyed success at FC Porto under Jose Mourinho. "I\'m here to win titles with Werder," the 22-year-old said after his first training session with his new club. "I like Bremen and would only have wanted to come here." Carlos Alberto started his career with Fluminense, and helped them to lift the Campeonato Carioca in 2002. In January 2004 he moved on to FC Porto, who were coached by José Mourinho, and the club won the Portuguese title as well as the Champions League. Early in 2005, he moved to Corinthians, where he impressed as they won the Brasileirão,but in 2006 Corinthians had a poor season and Carlos Alberto found himself at odds with manager, Emerson Leão. Their poor relationship came to a climax at a Copa Sul-Americana game against Club Atlético Lanús, and Carlos Alberto declared that he would not play for Corinthians again while Leão remained as manager. Since January this year he has been on loan with his first club Fluminense. Bundesliga champions VfB Stuttgart said on Sunday that they would sign a loan agreement with Real Zaragoza on Monday for Ewerthon, the third top Brazilian player to join the German league in three days. A VfB spokesman said Ewerthon, who played in the Bundesliga for Borussia Dortmund from 2001 to 2005, was expected to join the club for their pre-season training in Austria on Monday. On Friday, Ailton returned to Germany where he was the league\'s top scorer in 2004, signing a one-year deal with Duisburg on a transfer from Red Star Belgrade. E-mail to a friend .',
                        '(CNN) -- Football superstar, celebrity, fashion icon, multimillion-dollar heartthrob. Now, David Beckham is headed for the Hollywood Hills as he takes his game to U.S. Major League Soccer. CNN looks at how Bekham fulfilled his dream of playing for Manchester United, and his time playing for England. The world\'s famous footballer has begun a five-year contract with the Los Angeles Galaxy team, and on Friday Beckham will meet the press and reveal his new shirt number. This week, we take an in depth look at the life and times of Beckham, as CNN\'s very own "Becks," Becky Anderson, sets out to examine what makes the man tick -- as footballer, fashion icon and global phenomenon. It\'s a long way from the streets of east London to the Hollywood Hills and Becky charts Beckham\'s incredible rise to football stardom, a journey that has seen his skills grace the greatest stages in world soccer. She goes in pursuit of the current hottest property on the sports/celebrity circuit in the U.S. and along the way explores exactly what\'s behind the man with the golden boot. CNN will look back at the life of Beckham, the wonderfully talented youngster who fulfilled his dream of playing for Manchester United, his marriage to pop star Victoria, and the trials and tribulations of playing for England. We\'ll look at the highs (scoring against Greece), the lows (being sent off during the World Cup), the Man. U departure for the Galacticos of Madrid -- and now the Home Depot stadium in L.A. We\'ll ask how Beckham and his family will adapt to life in Los Angeles -- the people, the places to see and be seen and the celebrity endorsement. Beckham is no stranger to exposure. He has teamed with Reggie Bush in an Adidas commercial, is the face of Motorola, is the face on a PlayStation game and doesn\'t need fashion tips as he has his own international clothing line. But what does the star couple need to do to become an accepted part of Tinseltown\'s glitterati? The road to major league football in the U.S.A. is a well-worn route for some of the world\'s greatest players. We talk to some of the former greats who came before him and examine what impact these overseas stars had on U.S. soccer and look at what is different now. We also get a rare glimpse inside the David Beckham academy in L.A, find out what drives the kids and who are their heroes. The perception that in the U.S.A. soccer is a "game for girls" after the teenage years is changing. More and more young kids are choosing the European game over the traditional U.S. sports. E-mail to a friend .',
                        'LOS ANGELES, California (CNN) -- Youssif, the 5-year-old burned Iraqi boy, rounded the corner at Universal Studios when suddenly the little boy hero met his favorite superhero. Youssif has always been a huge Spider-Man fan. Meeting him was "my favorite thing," he said. Spider-Man was right smack dab in front of him, riding a four-wheeler amid a convoy of other superheroes. The legendary climber of buildings and fighter of evil dismounted, walked over to Youssif and introduced himself. Spidey then gave the boy from a far-away land a gentle hug, embracing him in his iconic blue and red tights. He showed Youssif a few tricks, like how to shoot a web from his wrist. Only this time, no web was spun. "All right Youssif!" Spider-Man said after the boy mimicked his wrist movement. Other superheroes crowded around to get a closer look. Even the Green Goblin stopped his villainous ways to tell the boy hi. Youssif remained unfazed. He didn\'t take a liking to Spider-Man\'s nemesis. Spidey was just too cool. "It was my favorite thing," the boy said later. "I want to see him again." He then felt compelled to add: "I know it\'s not the real Spider-Man." This was the day of dreams when the boy\'s nightmares were, at least temporarily, forgotten. He met SpongeBob, Lassie and a 3-year-old orangutan named Archie. The hairy, brownish-red primate took to the boy, grabbing his hand and holding it. Even when Youssif pulled away, Archie would inch his hand back toward the boy\'s and then snatch it. See Youssif enjoy being a boy again » . The boy giggled inside a play area where sponge-like balls shot out of toy guns. It was a far different artillery than what he was used to seeing in central Baghdad, as recently as a week ago. He squealed with delight and raced around the room collecting as many balls as he could. He rode a tram through the back stages at Universal Studios. At one point, the car shook. Fire and smoke filled the air, debris cascaded down and a big rig skidded toward the vehicle. The boy and his family survived the pretend earthquake unscathed. "Even I was scared," the dad said. "Well, I wasn\'t," Youssif replied. The father and mother grinned from ear to ear throughout the day. Youssif pushed his 14-month-old sister, Ayaa, in a stroller. "Did you even need to ask us if we were interested in coming here?" Youssif\'s father said in amazement. "Other than my wedding day, this is the happiest day of my life," he said. Just a day earlier, the mother and father talked about their journey out of Iraq and to the United States. They also discussed that day nine months ago when masked men grabbed their son outside the family home, doused him in gas and set him on fire. His mother heard her boy screaming from inside. The father sought help for his boy across Baghdad, but no one listened. He remembers his son\'s two months of hospitalization. The doctors didn\'t use anesthetics. He could hear his boy\'s piercing screams from the other side of the hospital. Watch Youssif meet his doctor and play with his little sister » . The father knew that speaking to CNN would put his family\'s lives in jeopardy. The possibility of being killed was better than seeing his son suffer, he said. "Anything for Youssif," he said. "We had to do it." They described a life of utter chaos in Baghdad. Neighbors had recently given birth to a baby girl. Shortly afterward, the father was kidnapped and killed. Then, there was the time when some girls wore tanktops and jeans. They were snatched off the street by gunmen. The stories can be even more gruesome. The couple said they had heard reports that a young girl was kidnapped and beheaded --and her killers sewed a dog\'s head on the corpse and delivered it to her family\'s doorstep. "These are just some of the stories," said Youssif\'s mother, Zainab. Under Saddam Hussein, there was more security and stability, they said. There was running water and electricity most of the time. But still life was tough under the dictator, like the time when Zainab\'s uncle disappeared and was never heard from again after he read a "religious book," she said. Sitting in the parking lot of a Target in suburban Los Angeles, Youssif\'s father watched as husbands and wives, boyfriends and girlfriends, parents and their children, came and went. Some held hands. Others smiled and laughed. "Iraq finished," he said in what few English words he knows. He elaborated in Arabic: His homeland won\'t be enjoying such freedoms anytime soon. It\'s just not possible. Too much violence. Too many killings. His two children have only seen war. But this week, the family has seen a much different side of America -- an outpouring of generosity and a peaceful nation at home. "It\'s been a dream," the father said. He used to do a lot of volunteer work back in Baghdad. "Maybe that\'s why I\'m being helped now," the father said. At Universal Studios, he looked out across the valley below. The sun glistened off treetops and buildings. It was a picturesque sight fit for a Hollywood movie. "Good America, good America," he said in English. E-mail to a friend . CNN\'s Arwa Damon contributed to this report.'
]

cnn_daily_article_highlights = ['Werder Bremen pay a club record $10.7 million for Carlos Alberto .\nThe Brazilian midfielder won the Champions League with FC Porto in 2004 .\nSince January he has been on loan with his first club, Fluminense .',
                                'Beckham has agreed to a five-year contract with Los Angeles Galaxy .\nNew contract took effect July 1, 2007 .\nFormer English captain to meet press, unveil new shirt number Friday .\nCNN to look at Beckham as footballer, fashion icon and global phenomenon .',
                                'Boy on meeting Spider-Man: "It was my favorite thing"\nYoussif also met SpongeBob, Lassie and an orangutan at Universal Studios .\nDad: "Other than my wedding day, this is the happiest day of my life"'
]

cnn_df = pd.DataFrame({"articles":cnn_daily_articles, "highligths":cnn_daily_article_highlights})

cnn_df.head()

In [ ]:
article1_embedding    = client.embeddings.create(input=cnn_df.articles.iloc[0], model=model).data[0].embedding
article2_embedding    = client.embeddings.create(input=cnn_df.articles.iloc[1], model=model).data[0].embedding
article3_embedding    = client.embeddings.create(input=cnn_df.articles.iloc[2], model=model).data[0].embedding

print(cosine_similarity(article1_embedding, article2_embedding))
print(cosine_similarity(article1_embedding, article3_embedding))

# References  
- [Azure Documentation - Azure OpenAI Models](https://learn.microsoft.com/azure/cognitive-services/openai/concepts/models?WT.mc_id=academic-105485-koreyst)  
- [OpenAI Studio Examples](https://oai.azure.com/portal?WT.mc_id=academic-105485-koreyst)  


# For More Help  
[OpenAI 상용화 팀](AzureOpenAITeam@microsoft.com)


# Contributors
* Louis Li  


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**면책 조항**:  
이 문서는 AI 번역 서비스 [Co-op Translator](https://github.com/Azure/co-op-translator)를 사용하여 번역되었습니다. 정확성을 위해 최선을 다하고 있으나, 자동 번역에는 오류나 부정확한 부분이 있을 수 있음을 유의하시기 바랍니다. 원문 문서가 권위 있는 출처로 간주되어야 합니다. 중요한 정보의 경우 전문적인 인간 번역을 권장합니다. 본 번역 사용으로 인한 오해나 잘못된 해석에 대해 당사는 책임을 지지 않습니다.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
